<a href="https://colab.research.google.com/github/FKz11/NLP/blob/main/lesson_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson-12

Взять датасет
https://huggingface.co/datasets/merionum/ru_paraphraser
решить задачу парафраза

### Библиотеки:

In [ ]:
!pip install -qq datasets

     |████████████████████████████████| 431 kB 13.3 MB/s 
     |████████████████████████████████| 212 kB 60.8 MB/s 
     |████████████████████████████████| 115 kB 63.5 MB/s 
     |████████████████████████████████| 127 kB 51.5 MB/s 


In [ ]:
!pip install -q transformers

     |████████████████████████████████| 4.9 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 52.0 MB/s 


In [ ]:
import numpy as np
import pandas as pd

import torch
from torch.utils.data import DataLoader, Dataset

from datasets import load_dataset

from transformers import GPT2LMHeadModel
from transformers import GPT2Tokenizer

from transformers import DataCollatorForLanguageModeling, TextDataset, TrainingArguments, Trainer

### Данные:

In [ ]:
dataset = load_dataset("merionum/ru_paraphraser")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-aad01005423eb70c/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
train = dataset['train']
test = dataset['test']
train

Dataset({
    features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
    num_rows: 7227
})

1: precise paraphrases

0: near paraphrases

-1: non-paraphrases

In [ ]:
threshold = 0
train_x = np.array(train['text_1'])[np.array(train['class']).astype(int)>threshold]
train_y = np.array(train['text_2'])[np.array(train['class']).astype(int)>threshold]
test_x = np.array(test['text_1'])[np.array(test['class']).astype(int)>threshold]
test_y = np.array(test['text_2'])[np.array(test['class']).astype(int)>threshold]
train_x[0], train_y[0]

('Приставы соберут отпечатки пальцев российских должников.',
 'Приставы снимут отпечатки пальцев у злостных неплательщиков.')

Тоесть берём только точные paraphrases

Так как оценка будет визуальной, то для теста возьмём только 10 примеров

In [ ]:
df = pd.DataFrame({'orig': list(train_x)+list(test_x)[:-10], 'modern': list(train_y)+list(test_y)[:-10]})
df.head()

,orig,modern
0,Приставы соберут отпечатки пальцев российских ...,Приставы снимут отпечатки пальцев у злостных н...
1,Москвичи смогут забронировать в Интернете мест...,В Москве можно будет забронировать место на кл...
2,Северокорейский лидер впервые за 19 лет поздра...,Лидер КНДР впервые за 19 лет поздравил согражд...
3,Мужчина из Подмосковья случайно убил жену в Но...,Житель Подмосковья случайно убил жену на новог...
4,Житель Украины расстрелял посетителей кафе.,На Украине мужчина через окно расстрелял посет...


In [ ]:
test=list(zip(list(test_x)[-10:], list(test_y)[-10:]))
test[0]

('СК: ошибка пилота могла стать причиной падения самолета на Ставрополье',
 'СК: На Ставрополье самолет мог упасть из-за ошибки пилота')

Добавим спец символы, обозначающие начало и конецфраз, а так же их разделение

In [ ]:
df['combined']='<s>'+df['orig']+'</s>'+'>>>>'+'<p>'+df['modern']+'</p>'
df.head()

,orig,modern,combined
0,Приставы соберут отпечатки пальцев российских ...,Приставы снимут отпечатки пальцев у злостных н...,<s>Приставы соберут отпечатки пальцев российск...
1,Москвичи смогут забронировать в Интернете мест...,В Москве можно будет забронировать место на кл...,<s>Москвичи смогут забронировать в Интернете м...
2,Северокорейский лидер впервые за 19 лет поздра...,Лидер КНДР впервые за 19 лет поздравил согражд...,<s>Северокорейский лидер впервые за 19 лет поз...
3,Мужчина из Подмосковья случайно убил жену в Но...,Житель Подмосковья случайно убил жену на новог...,<s>Мужчина из Подмосковья случайно убил жену в...
4,Житель Украины расстрелял посетителей кафе.,На Украине мужчина через окно расстрелял посет...,<s>Житель Украины расстрелял посетителей кафе....


Сохраним в текстовый файл наши данные

In [ ]:
df.combined.to_csv('combined.txt', sep='\n', index=False)

### Модель

Используем gpt2 модель, которую проходили на вебинаре

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2", eos_token='</p>')
ru_gpt = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2", output_attentions=True)

Downloading:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/551M [00:00<?, ?B/s]

In [ ]:
ru_gpt.to('cuda:0');

Подготовка к тренировке модели

In [ ]:
data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
train_path='/content/combined.txt'
train_dataset=TextDataset(tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


Настройка параметров обучения

In [ ]:
training_args = TrainingArguments(
    output_dir="./gpt2-chief", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 100, # Number of update steps between two evaluations.
    save_steps=100, # after # steps model is saved
    warmup_steps=100,# number of warmup steps for learning rate scheduler
    logging_steps=100
    )

In [ ]:
trainer = Trainer(
    model=ru_gpt,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator
)

Но сначала посмотрим, что модель генерирует без обучения

In [ ]:
def paraphrase(text, repetition_penalty=5., temperature=0.5, num_beams=10):
  ru_gpt.eval()
  spec_text = '<s>'+text+'</s>'+'>>>>'+'<p>'
  tokens = tokenizer(spec_text, return_tensors='pt').to('cuda:0')
  size = tokens['input_ids'].shape[1]
  output = ru_gpt.generate(
    **tokens,
    do_sample=False,
    max_length=size+50, 
    repetition_penalty=repetition_penalty, 
    temperature=temperature,
    num_beams=num_beams,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id
  )

  decoded = tokenizer.decode(output[0])
  result = decoded[len(spec_text):]
  stop_indx = result.index('</p>') if '</p>' in result else None
  result = result[:stop_indx]
  return result

In [ ]:
for text in test[:3]:
  print(f'Input: {text[0]}')
  print(f'Paraphrase: {paraphrase(text[0], repetition_penalty=5., temperature=0.5, num_beams=10)}')
  print(f'True: {text[1]}')
  print()

Input: СК: ошибка пилота могла стать причиной падения самолета на Ставрополье
Paraphrase: <div class="container" style="border: 1px solid #ccc;"> <a href="http://www.liveinternet.ru/...✂" target="_blank" title="
True: СК: На Ставрополье самолет мог упасть из-за ошибки пилота

Input: СМИ: Россия передала ЕС свой санкционный "черный список"
Paraphrase: Российские СМИ сообщают о том, что Европейский союз (ЕС) принял решение не продлевать санкции против России в связи с ситуацией на Украине.  Об этом сообщает РИА Новости со ссылкой на пресс-службу Еврокомиссии.
True: МИД России передал в представительство ЕС список персон нон грата

Input: Медведев о назначении Саакашвили: шапито-шоу продолжается
Paraphrase: 
http://www.gazeta.ru/culture/news/2015/12/27/n_14623949.shtml
[1] http://www.kommersant.ru/
True: Медведев назвал назвачение Саакашвили одесским губернатором шапито-шоу



Как видно, модель совсем необучена на задачу Paraphrase

Тренируем модель 

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 631
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 474


Step,Training Loss
100,2.361200
200,1.638800
300,1.424200
400,1.217500


Saving model checkpoint to ./gpt2-chief/checkpoint-100
Configuration saved in ./gpt2-chief/checkpoint-100/config.json
Model weights saved in ./gpt2-chief/checkpoint-100/pytorch_model.bin
Saving model checkpoint to ./gpt2-chief/checkpoint-200
Configuration saved in ./gpt2-chief/checkpoint-200/config.json
Model weights saved in ./gpt2-chief/checkpoint-200/pytorch_model.bin
Saving model checkpoint to ./gpt2-chief/checkpoint-300
Configuration saved in ./gpt2-chief/checkpoint-300/config.json
Model weights saved in ./gpt2-chief/checkpoint-300/pytorch_model.bin
Saving model checkpoint to ./gpt2-chief/checkpoint-400
Configuration saved in ./gpt2-chief/checkpoint-400/config.json
Model weights saved in ./gpt2-chief/checkpoint-400/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=474, training_loss=1.5807346231323758, metrics={'train_runtime': 112.6649, 'train_samples_per_second': 16.802, 'train_steps_per_second': 4.207, 'total_flos': 123656454144000.0, 'train_loss': 1.5807346231323758, 'epoch': 3.0})

Посмотрим какие теперь результаты:

In [ ]:
for text in test[:3]:
  print(f'Input: {text[0]}')
  print(f'Paraphrase: {paraphrase(text[0], repetition_penalty=5., temperature=0.5, num_beams=10)}')
  print(f'True: {text[1]}')
  print()

Input: СК: ошибка пилота могла стать причиной падения самолета на Ставрополье
Paraphrase: Самолет упал на Ставрополье из-за ошибки пилотов 
True: СК: На Ставрополье самолет мог упасть из-за ошибки пилота

Input: СМИ: Россия передала ЕС свой санкционный "черный список"
Paraphrase: Россия передала Евросоюзу «черный список» 
True: МИД России передал в представительство ЕС список персон нон грата

Input: Медведев о назначении Саакашвили: шапито-шоу продолжается
Paraphrase: Путин назвал Шапито-шоу «шапито-шоу» 
True: Медведев назвал назвачение Саакашвили одесским губернатором шапито-шоу



Результаты очень даже неплохие

Попробуем улучшить генерацию увеличивая параметр beem

In [ ]:
for text in test[:3]:
  print(f'Input: {text[0]}')
  print(f'Paraphrase: {paraphrase(text[0], repetition_penalty=5., temperature=0.5, num_beams=100)}')
  print(f'True: {text[1]}')
  print()

Input: СК: ошибка пилота могла стать причиной падения самолета на Ставрополье
Paraphrase: Пассажирский самолет упал на Ставрополье 
True: СК: На Ставрополье самолет мог упасть из-за ошибки пилота

Input: СМИ: Россия передала ЕС свой санкционный "черный список"
Paraphrase: Россия передала Евросоюзу «черный список» 
True: МИД России передал в представительство ЕС список персон нон грата

Input: Медведев о назначении Саакашвили: шапито-шоу продолжается
Paraphrase: Путин назвал Шапито-шоу «шапито-шоу» 
True: Медведев назвал назвачение Саакашвили одесским губернатором шапито-шоу



Теперь попробуем уменьшить температуру

In [ ]:
for text in test[:3]:
  print(f'Input: {text[0]}')
  print(f'Paraphrase: {paraphrase(text[0], repetition_penalty=5., temperature=0.1, num_beams=100)}')
  print(f'True: {text[1]}')
  print()

Input: СК: ошибка пилота могла стать причиной падения самолета на Ставрополье
Paraphrase: Пассажирский самолет упал на Ставрополье 
True: СК: На Ставрополье самолет мог упасть из-за ошибки пилота

Input: СМИ: Россия передала ЕС свой санкционный "черный список"
Paraphrase: Россия передала Евросоюзу «черный список» 
True: МИД России передал в представительство ЕС список персон нон грата

Input: Медведев о назначении Саакашвили: шапито-шоу продолжается
Paraphrase: Путин назвал Шапито-шоу «шапито-шоу» 
True: Медведев назвал назвачение Саакашвили одесским губернатором шапито-шоу



Генерации не сильно поменялись, но они и так очень даже хорошие.

Результат:

In [ ]:
for text in test:
  print(f'Input: {text[0]}')
  print(f'Paraphrase: {paraphrase(text[0], repetition_penalty=5., temperature=0.5, num_beams=10)}')
  print(f'True: {text[1]}')
  print()

Input: СК: ошибка пилота могла стать причиной падения самолета на Ставрополье
Paraphrase: Самолет упал на Ставрополье из-за ошибки пилотов 
True: СК: На Ставрополье самолет мог упасть из-за ошибки пилота

Input: СМИ: Россия передала ЕС свой санкционный "черный список"
Paraphrase: Россия передала Евросоюзу «черный список» 
True: МИД России передал в представительство ЕС список персон нон грата

Input: Медведев о назначении Саакашвили: шапито-шоу продолжается
Paraphrase: Путин назвал Шапито-шоу «шапито-шоу» 
True: Медведев назвал назвачение Саакашвили одесским губернатором шапито-шоу

Input: Трое зрителей погибли во время авторалли в Эстонии
Paraphrase: В Эстонии четверо человек погибли при крушении легкомоторного самолета 
True: Трое зрителей погибли в ходе авторалли в Эстонии

Input: Аракчи: очередная встреча "шестерки" и Ирана состоится в Вене 4 июня
Paraphrase: Встреча «шестерки» и Ирана пройдет в Вене 4 июня 
True: Новые переговоры Ирана и "шестерки" состоятся в Вене 4 июня

Input: 

Как видно, предложения выглядят естественно и цельно, но не всегда попадают в суть.

На самом деле gpt состоит только из декодера, что для данной задачи не очень хорошо, так как мы хотим не придумывать совсем что то новое, а больше ориентироваться на входные данные. 

Поэтому в этой задаче лучше подойдёт такая архитектура как Т5. В ней есть и энкодер и декодер.

Возьмём такую модель уже обученную на задаче paraphrase на русском языке:

In [ ]:
!pip install -q transformers[SentencePiece]

     |████████████████████████████████| 4.9 MB 34.8 MB/s 
     |████████████████████████████████| 6.6 MB 61.8 MB/s 
     |████████████████████████████████| 163 kB 69.4 MB/s 
     |████████████████████████████████| 1.3 MB 64.0 MB/s 


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("cointegrated/rut5-base-paraphraser")
ru_t5 = T5ForConditionalGeneration.from_pretrained("cointegrated/rut5-base-paraphraser", output_attentions=True)

Downloading:   0%|          | 0.00/828k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/315 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/724 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/977M [00:00<?, ?B/s]

In [ ]:
ru_t5.to('cuda:0');

In [ ]:
def paraphrase(text, repetition_penalty=5., temperature=0.5, num_beams=10):
  ru_t5.eval()
  spec_text = '<s>'+text
  tokens = tokenizer(spec_text, return_tensors='pt').to('cuda:0')
  size = tokens['input_ids'].shape[1]
  output = ru_t5.generate(
    **tokens,
    do_sample=False,
    max_length=size+50, 
    repetition_penalty=repetition_penalty, 
    temperature=temperature,
    num_beams=num_beams
  )

  result = tokenizer.decode(output[0])
  pad_indx = result.index('<pad>')+5 if '<pad>' in result else None
  stop_indx = result.index('</s>') if '</s>' in result else None
  result = result[pad_indx:stop_indx]
  return result

In [ ]:
for text in test:
  print(f'Input: {text[0]}')
  print(f'Paraphrase: {paraphrase(text[0], repetition_penalty=5., temperature=0.5, num_beams=10)}')
  print(f'True: {text[1]}')
  print()

Input: СК: ошибка пилота могла стать причиной падения самолета на Ставрополье
Paraphrase:  Названа возможная причина падения самолета на Ставрополье
True: СК: На Ставрополье самолет мог упасть из-за ошибки пилота

Input: СМИ: Россия передала ЕС свой санкционный "черный список"
Paraphrase:  СМИ: Россия передала ЕС санкционный "черный список"
True: МИД России передал в представительство ЕС список персон нон грата

Input: Медведев о назначении Саакашвили: шапито-шоу продолжается
Paraphrase:  В Кремле прокомментировали кандидатуру Саакашвили на пост премьера
True: Медведев назвал назвачение Саакашвили одесским губернатором шапито-шоу

Input: Трое зрителей погибли во время авторалли в Эстонии
Paraphrase:  Во время авторалли в Эстонии погибли три человека
True: Трое зрителей погибли в ходе авторалли в Эстонии

Input: Аракчи: очередная встреча "шестерки" и Ирана состоится в Вене 4 июня
Paraphrase:  В Вене пройдет очередная встреча "шестерки" и Ирана
True: Новые переговоры Ирана и "шестерки" с

Как видно, это модель намного точнее, но при этом менее вариативнее.